In [1]:
# importing libraries
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image
import cv2

In [2]:
mtcnn = MTCNN(image_size=240, margin=0, min_face_size=20) # initializing mtcnn for face detection
resnet = InceptionResnetV1(pretrained='vggface2').eval() # initializing resnet for face img to embeding conversion

dataset=datasets.ImageFolder('D:\XIS Work\Facenet Project\photos') # photos folder path 
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # accessing names of peoples from folder names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = [] # list of cropped faces from photos folder
name_list = [] # list of names corrospoing to cropped photos
embedding_list = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using resnet

for img, idx in loader:
    face, prob = mtcnn(img, return_prob=True) 
    if face is not None and prob>0.90: # if face detected and porbability > 90%
        emb = resnet(face.unsqueeze(0)) # passing cropped face into resnet model to get embedding matrix
        embedding_list.append(emb.detach()) # resulten embedding matrix is stored in a list
        name_list.append(idx_to_class[idx]) # names are stored in a list

In [3]:
data = [embedding_list, name_list]
torch.save(data, 'data.pt') # saving data.pt file

In [18]:
# def face_match(img_path, data_path): 
#     # img_path= location of photo, data_path= location of data.pt 
#     img = cv2.imread(img_path)  # Read image using OpenCV
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

#     # Resize the image to 512x512
#     img = cv2.resize(img, (512, 512))

#     face, prob = mtcnn(img, return_prob=True) # returns cropped face and probability
#     emb = resnet(face.unsqueeze(0)).detach() # detach is to make required gradient false
    
#     saved_data = torch.load(data_path) # loading data.pt file
#     embedding_list = saved_data[0] # getting embedding data
#     name_list = saved_data[1] # getting list of names
#     dist_list = [] # list of matched distances, minimum distance is used to identify the person
    
#     for idx, emb_db in enumerate(embedding_list):
#         dist = torch.dist(emb, emb_db).item()
#         dist_list.append(dist)
        
#     idx_min = dist_list.index(min(dist_list))
    
#     # Set the threshold for considering a match
#     threshold = 0.8
    
#     if min(dist_list) < threshold:
#         result = (name_list[idx_min], min(dist_list), 'Match')
#     else:
#         result = ('Unknown', min(dist_list), 'No Match')

#     # Overlay text on the image
#     # text = f'Face matched with: {result[0]} With distance: {result[1]:.4f} Status: {result[2]}'
#     text = f'Face matched with: {result[0]}'
#     cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

#     # Create a window with a specific size
#     cv2.namedWindow('Face Match Result', cv2.WINDOW_NORMAL)
#     cv2.resizeWindow('Face Match Result', 512, 512)

#     # Display the image
#     cv2.imshow('Face Match Result', img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

#     return result

# result = face_match('afi.png', 'data.pt')

In [4]:
def face_match(camera, data_path): 
    cap = cv2.VideoCapture(camera)
    while True:
        ret, frames = cap.read()
        img = cv2.cvtColor(frames, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (512, 512))

        face, prob = mtcnn(img, return_prob=True) # returns cropped face and probability
        
        # Check if a face is detected
        if face is not None:
            emb = resnet(face.unsqueeze(0)).detach() # detach is to make required gradient false
            
            saved_data = torch.load(data_path) # loading data.pt file
            embedding_list = saved_data[0] # getting embedding data
            name_list = saved_data[1] # getting list of names
            dist_list = [] # list of matched distances, minimum distance is used to identify the person
            
            for idx, emb_db in enumerate(embedding_list):
                dist = torch.dist(emb, emb_db).item()
                dist_list.append(dist)
                
            idx_min = dist_list.index(min(dist_list))
            
            # Set the threshold for considering a match
            threshold = 0.8
            
            if min(dist_list) < threshold:
                result = (name_list[idx_min], min(dist_list), 'Match')
            else:
                result = ('Unknown', min(dist_list), 'No Match')

            # Overlay text on the image
            text = f'Face matched with: {result[0]}'
            cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

        # Create a window with a specific size
        cv2.namedWindow('Face Match Result', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Face Match Result', 512, 512)

        # Display the image
        cv2.imshow('Face Match Result', img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    return result

result = face_match(0, 'data.pt')

In [ ]:
#.py code

# # importing libraries
# from facenet_pytorch import MTCNN, InceptionResnetV1
# import torch
# from torchvision import datasets
# from torch.utils.data import DataLoader
# from PIL import Image
# import cv2

# mtcnn = MTCNN(image_size=240, margin=0, min_face_size=20) # initializing mtcnn for face detection
# resnet = InceptionResnetV1(pretrained='vggface2').eval() # initializing resnet for face img to embeding conversion

# def face_match(img_path, data_path): 
#     # img_path= location of photo, data_path= location of data.pt 
#     img = cv2.imread(img_path)  # Read image using OpenCV
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

#     # Resize the image to 512x512
#     img = cv2.resize(img, (512, 512))

#     face, prob = mtcnn(img, return_prob=True) # returns cropped face and probability
#     emb = resnet(face.unsqueeze(0)).detach() # detach is to make required gradient false
    
#     saved_data = torch.load(data_path) # loading data.pt file
#     embedding_list = saved_data[0] # getting embedding data
#     name_list = saved_data[1] # getting list of names
#     dist_list = [] # list of matched distances, minimum distance is used to identify the person
    
#     for idx, emb_db in enumerate(embedding_list):
#         dist = torch.dist(emb, emb_db).item()
#         dist_list.append(dist)
        
#     idx_min = dist_list.index(min(dist_list))
    
#     # Set the threshold for considering a match
#     threshold = 0.8
    
#     if min(dist_list) < threshold:
#         result = (name_list[idx_min], min(dist_list), 'Match')
#     else:
#         result = ('Unknown', min(dist_list), 'No Match')

#     # Overlay text on the image
#     # text = f'Face matched with: {result[0]} With distance: {result[1]:.4f} Status: {result[2]}'
#     text = f'Face matched with: {result[0]}'
#     cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

#     # Create a window with a specific size
#     cv2.namedWindow('Face Match Result', cv2.WINDOW_NORMAL)
#     cv2.resizeWindow('Face Match Result', 512, 512)

#     # Display the image
#     cv2.imshow('Face Match Result', img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

#     return result

# result = face_match('afi.png', 'data.pt')

#---------------------------------------------------------------------------------